# Virtual Private Server - On Demend
The pupose of this notebook is to allow a remote /ideally/ customisable and persistent environment for executing code whilst not at your personal computer. This is achieved through the use of Googles generous access to Google Colab remote machines and this notebook to generate an SSH-able VPS to access your google drive files and system comfigurations providing easier access to persistent research.

## Requirements.
The are a few requirements that will need to be developed in order to maximise the effectiveness of this system.
1. A Google drive account to access datasets and code.
1. An ssh client available on your PC (for those with limited computer access there are SSH clients available via browser extensions).
1. Ideally a Nix or Makefile build script located on your drive as this will be required to generate your Personal Computing Environment (PCE).

## The Methodology.
1. Access Google Drive. 
1. Confiure SSH & SSH Keys. (Required for port forwarding as an alternatvie to ngrok).
1. Install the SSHD service.
1. Configure a root password (to log in with).
1. Start the SSHD service.
1. Create the SSH port forwarding tunnel.
1. Print the access information to the screen.

Finally, Copy and paste the access information into your ssh client and you're good to go.

### Step 1.
Mounting google drive to the Google Colab Instance.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

/Optional code to confirm it has mounted correctly/

In [ ]:
!ls '/content/gdrive/MyDrive/'

### Step 2.
Configuring SSH access and SSH Keys to connect.

In [ ]:
# Generate the SSH key.
!ssh-keygen -t ed25519 -C "general@email.com"

Start the ssh-agent and add the relavent identity to the system. (default ssh id is assumed here but may be different).

In [ ]:
# Start the ssd agent and add the ssh Identify. 
!eval "$(ssh-agent ssh-add "/root/.ssh/id_ed25519")"

### Step 3. 
Set up sshd 

In [ ]:
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

### Step 4.
Set up a root password.

In [ ]:
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#### Step 5.
Run sshd.

In [ ]:
#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

In [ ]:
#### Step 6.
Create the SSH tunnel.


In [ ]:
ssh -o StrictHostKeyChecking=no -R 22:localhost ssh.localhost.run

#### Step 7.
Get access details.

In [ ]:
#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')